In [ ]:
#必要的库
pip install transformers
pip install evaluate
pip install datasets

In [ ]:
#java数据集生成
#对应“java”文件夹
import csv
#sstubs.csv的文件位置
csv_path='.\datasets\java\sstubs-miner\sstubs.csv'
#保存数据集的位置
output_buggy='.\java.buggy'
output_fixxed='.\java.fixed'

csv.field_size_limit(500 * 1024 * 1024)
with open(csv_path,'r',encoding='utf-8') as f,open (output_buggy,'w',encoding='utf-8') as buggy ,open (output_fixxed,'w',encoding='utf-8') as fixed:
  csv_reader = csv.DictReader(f)
  for line in csv_reader:
    if line['bug_source']!='' and line['fix_source']!='fix_source':
      line['bug_source']=line['bug_source'].replace('\n','')
      line['bug_source']=line['bug_source'].replace('\r','')
      line['fix_source']=line['fix_source'].replace('\n','')
      line['fix_source']=line['fix_source'].replace('\r','')
      buggy.write(line['bug_source']+'\n')
      fixed.write(line['fix_source']+'\n')

In [ ]:
#java_codesearchnet数据集生成
#对应“java_codesearchnet”文件夹
import os
import json
jsonl_path='.\datasets\java_codesearchnet'
jsonl_files = os.listdir(jsonl_path)
output_file='.\java.code.codesearchnet'
output_file=open(output_file,'w',encoding='utf-8')
for jsonl_file in jsonl_files:
  if os.path.splitext(jsonl_file)[1]  != '.jsonl':
    continue
  f=open(jsonl_file,'r',encoding='utf-8')
  code_lines=f.readlines()
  print(jsonl_file,len(code_lines))
  for code_line in code_lines:
    line_json=json.loads(code_line)
    code_tokens=line_json['code_tokens']
    code=' '.join(code_tokens)
    code=code.replace('\n','')
    code=code.replace('\r','')
    output_file.write(code+'\n')
  f.close()
output_file.close()

In [ ]:
#java_long数据集生成
#对应“java_long”文件夹
import pickle
java_long_pkl0=open(r'.\datasets\java_long\data0.pkl','rb')
java_long_pkl1=open(r'.\datasets\java_long\data1.pkl','rb')
output_buggy='.\java.buggy.long'
output_fixxed='.\java.fixed.long'
pkl_content=pickle.load(java_long_pkl0)
with open (output_buggy,'w',encoding='utf-8') as buggy ,open (output_fixxed,'w',encoding='utf-8') as fixed:
  for example in pkl_content:
    buggy_code=example['old']
    fixed_code=example['new']
    buggy_code=buggy_code.replace('\n','')
    buggy_code=buggy_code.replace('\r','')
    fixed_code=fixed_code.replace('\n','')
    fixed_code=fixed_code.replace('\r','')
    buggy.write(buggy_code+'\n')
    fixed.write(fixed_code+'\n')

pkl_content=pickle.load(java_long_pkl1)
with open (output_buggy,'a',encoding='utf-8') as buggy ,open (output_fixxed,'a',encoding='utf-8') as fixed:
  for example in pkl_content:
    buggy_code=example['old']
    fixed_code=example['new']
    buggy_code=buggy_code.replace('\n','')
    buggy_code=buggy_code.replace('\r','')
    fixed_code=fixed_code.replace('\n','')
    fixed_code=fixed_code.replace('\r','')
    buggy.write(buggy_code+'\n')
    fixed.write(fixed_code+'\n')
pkl_content=None

In [ ]:
#java_mtufano数据集生成
#对应“java_mtufano”文件夹
import os
import json
#名如“000021ead7afe80b8eb1d34d61fbaf6d41f30555”等超多文件夹存放的目录
code_datasets_path='.\datasets\java_mtufano\bug-fixes-methods\sciclone\data10\mtufano\deepLearningMutants\out\changes\code'
#保存数据集的位置
output_buggy='.\java.buggy.mtufano'
output_fixed='.\java.fixed.mtufano'
output_file_buggy=open(output_buggy,'w',encoding='utf-8')
output_file_fixed=open(output_fixed,'w',encoding='utf-8')
top_dirs_array=os.listdir(code_datasets_path)
account_code=0
for top_dir in top_dirs_array:
  secondary_dirs_array=os.listdir(code_datasets_path+'\\'+top_dir)
  for secondary_dir in secondary_dirs_array:
    third_dirs_array=os.listdir(code_datasets_path+'\\'+top_dir+'\\'+secondary_dir)
    for third_dir in third_dirs_array:
      buggy_code_file_path=code_datasets_path+'\\'+top_dir+'\\'+secondary_dir+'\\'+third_dir+'\\before.java'
      fixed_code_file_path=code_datasets_path+'\\'+top_dir+'\\'+secondary_dir+'\\'+third_dir+'\\after.java'
      if not os.path.exists(buggy_code_file_path) or not os.path.exists(fixed_code_file_path):
        continue
      buggy_code_file=open(buggy_code_file_path,'r',encoding='utf-8')
      fixed_code_file=open(fixed_code_file_path,'r',encoding='utf-8')
      buggy_code=buggy_code_file.read()
      fixed_code=fixed_code_file.read()
      buggy_code=buggy_code.replace('\n','')
      buggy_code=buggy_code.replace('\r','')
      fixed_code=fixed_code.replace('\n','')
      fixed_code=fixed_code.replace('\r','')
      output_file_buggy.write(buggy_code+'\n')
      output_file_fixed.write(fixed_code+'\n')
      buggy_code_file.close()
      fixed_code_file.close()
      account_code=account_code+1
      if account_code%10000==0:
        print('Current progress:',account_code)

print(account_code)
output_file_buggy.close()
output_file_fixed.close()


In [ ]:
"""
对于上面得到的数据集，可自行挑选需要的部分组合，然后使用datasets_partitioning.ipynb拆分为训练集、验证集，用来预训练codebert和codegpt的交叉权重
需要修改下方的参数
#训练集文件位置
train_buggy_file='train.buggy-fixed.buggy'
train_fixed_file='train.buggy-fixed.fixed'
#验证集文件位置
valid_buggy_file='valid.buggy-fixed.buggy'
valid_fixed_file='valid.buggy-fixed.fixed'

训练完成后会在
codebert_weight_path='.\codebert'
codegpt_weight_path='.\codegpt'
指定的文件夹内生成两个权重文件以及对应的配置文件，将这4个文件分别用来替换后续aecm模型的codebert和codegpt初始权重和配置文件
"""